In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

# Function 'get_transition_matrix' and its test

In [2]:
test_vector = np.array([0, 1, 1, 1, 0, 0, 1])
answer = np.array([[1./3,2./3],[1./3,2./3]])

In [3]:
# plane_delays is a vector of 0s and 1s
def get_transition_matrix(plane_delays):
    mat = np.zeros((2,2))
    for i in range(0,len(plane_delays)-1):
        mat[plane_delays[i],plane_delays[i+1]] += 1
    return np.array([row/sum(row) for row in mat])

In [4]:
assert((answer == get_transition_matrix(test_vector)).all())
(answer == get_transition_matrix(test_vector)).all()

True

# Functions 'randomize_delays' and 
# 'get_pvalue_transition_matrix' and some testing

In [39]:
#plane_delays is np.array if it is only one series of delays or a list of np.arrays if it contains more. 
#It should be however from a single plane
def randomize_delays(plane_delays):
    if isinstance(plane_delays,np.ndarray):
        new_plane_delays = np.copy(plane_delays)
    elif isinstance(plane_delays,list):
        pass #this is needed when there are multilpe series for single plane
    np.random.shuffle(new_plane_delays)
    return new_plane_delays

In [85]:
#plane_delays is a list or numpy.array of plane delays, repetitions is number of randomization runs. 
#The smallest p-value is 2/repetitions
#Large p-value indicates a result is same as random. 
def get_pvalue_transition_matrix(plane_delays, repetitions=100):
    trans_mat = get_transition_matrix(plane_delays)
    vals_larger = np.zeros(trans_mat.shape)
    for i in range(0, repetitions):
        new_seq = randomize_delays(plane_delays)
        rand_trans_mat = get_transition_matrix(new_seq)
        #below is a 'method' that gives 1 if real is larger then random, 0.5 if they are equal and 0 if they are different
        vals_larger += (np.sign(trans_mat - rand_trans_mat)+1)/2
    vals_larger /= repetitions
    pvals = np.array([2*min(x,1-x) for x in np.nditer(vals_larger)]).reshape(vals_larger.shape)
    return pvals

In [ ]:
# Below is code for working

In [105]:
#test for get_pvalue_transition_matrix
np.random.seed(0)
test_vector = np.array([0, 1, 1, 1, 0, 0, 1])
expected_pvals = np.array([[0.91,0.91],[0.5,0.5]])

pvals = get_pvalue_transition_matrix(test_vector)

print(expected_pvals-pvals)
print(np.isclose(pvals,expected_pvals))

assert(np.isclose(pvals,expected_pvals).all())

[[0.00000000e+00 1.11022302e-16]
 [0.00000000e+00 0.00000000e+00]]
[[ True  True]
 [ True  True]]


In [25]:
test_vector2 = [test_vector, np.array([1,1,1,0,0,0,0,1,1])]
print(isinstance(test_vector,np.ndarray))
print(isinstance(test_vector2,list))
#new_plane_delays = np.copy(test_vector).reshape()
new_plane_delays = np.copy(test_vector)
np.random.shuffle(new_plane_delays)
print(test_vector)
print(new_plane_delays)

True
True
[0 1 1 1 0 0 1]
[1 0 1 0 1 0 1]


In [96]:
np.random.seed(0)
repetitions=100
trans_mat = get_transition_matrix(test_vector)
vals_larger = np.zeros(trans_mat.shape)
for i in range(0, repetitions):
    new_seq = randomize_delays(test_vector)
    rand_trans_mat = get_transition_matrix(new_seq)
    vals_larger += (np.sign(trans_mat - rand_trans_mat)+1)/2
vals_larger /= repetitions
print(vals_larger)
pvals = np.array([2*min(x,1-x) for x in np.nditer(vals_larger)]).reshape(vals_larger.shape)
print(pvals)

[[0.455 0.545]
 [0.25  0.75 ]]
[[0.91 0.91]
 [0.5  0.5 ]]


In [90]:
np.round(pvals*100)/100

array([[0.92, 0.92],
       [0.49, 0.49]])

In [106]:
?np.all